In [ ]:
import glob, json, os, sys

sys.path.append('/path/to/metrics/') # TODO: Replace with path to `SWE-bench/swebench/metrics` folder
from conversion import convert_log_to_ground_truth
from getters import get_logs_gold
from monitor import monitor_validation, monitor_logs_same_diff
sys.path = sys.path[:-1]

sys.path.append('/path/to/harness/') # TODO: Replace with path to `SWE-bench/swebench/harness` folder
from utils import has_attribute_or_import_error
sys.path = sys.path[:-1]

Declare repository; Fetch tasks, logs

In [ ]:
repo = 'pydicom/pydicom' # TODO: Replace with repository name
log_dir = '/logs/validation/pydicom' # TODO: Replace with path to folder of execution logs

Get map of version to setup commit

In [ ]:
tasks = json.load(open('path/to/pydicom-task-instances.json')) # TODO: Replace with path to versioned candidate task instances
tasks = sorted(tasks, key=lambda x: x['created_at'], reverse=True)
version_to_setup_commit = {}
for t in tasks:
    if 'version' in t and t['version'] not in version_to_setup_commit:
        version_to_setup_commit[t['version']] = t['base_commit']
assert(
    sorted(list([x or "" for x in version_to_setup_commit.keys()])) ==
    sorted(list(set([t['version'] or "" for t in tasks if 'version' in t])))
)
tasks = {t['instance_id']: t for t in tasks}

#### Monitor Validation

In [ ]:
failed_install, corrupt_test_patch, corrupt_patch, timeout, success = monitor_validation(log_dir)

In [ ]:
logs_same, logs_diff = monitor_logs_same_diff(log_dir)
print(f"Logs same: {len(logs_same)}")
print(f"Logs diff: {len(logs_diff)}")

#### Get [FP]2[FP] Tests

In [ ]:
inst_id_to_gt = {}
for d in logs_diff:
    status_gt = convert_log_to_ground_truth(d[0])
    inst_id_to_gt[d[0]] = status_gt

#### Create Task Instances `.json` file

In [ ]:
tasks_final = []
get_id_from_log = lambda x: x.split('/')[-1].split('.')[0]
for k, v in inst_id_to_gt.items():
    if len(v['FAIL_TO_PASS']) == 0:
        continue
    task = tasks[get_id_from_log(k)]
    task['FAIL_TO_PASS'] = v['FAIL_TO_PASS']
    task['PASS_TO_PASS'] = v['PASS_TO_PASS']
    task['environment_setup_commit'] = version_to_setup_commit[task['version']]

    # Do not consider tasks where the log before the patch has an attribute/import error
    log_path = os.path.join(log_dir, f'{t["repo"].split("/")[-1]}/{t["instance_id"]}.log')
    log_before, log_after = get_logs_gold(log_path)
    if has_attribute_or_import_error(log_before):
        continue

    tasks_final.append(task)
print(f"Final number of task instances: ", len(tasks_final))

SAVE_PATH = "path/to/save/tasks/to.json" # TODO: Replace this with a path to a .json file to save the task instances
with open(SAVE_PATH, 'w') as f:
    json.dump(tasks_final, fp=f)